In [39]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency
from sklearn.impute import SimpleImputer


In [40]:
train = pd.read_csv('train.csv')
print('Number of rows (Train): ' + str(len(train)))
print('Number of DUPLICATE rows (Train): ' + str(len(train) - len(train.drop_duplicates())))

train.head()

Number of rows (Train): 891
Number of DUPLICATE rows (Train): 0


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


Exploratory Data Analysis

In [41]:
# Check number of unique values for every column
for col in train.columns:
    print(col + ': ' + str(train[col].nunique()) + ' unique values')
print('_'*80)
print(train.info())
print('_'*80)
print(train.describe())
print('_'*80)
print(train.describe(include=['O']))
print('_'*80)
# Check for missing values in every column
print('Number of missing value(s) in every column (Train):')
print(train.isnull().sum())

PassengerId: 891 unique values
Survived: 2 unique values
Pclass: 3 unique values
Name: 891 unique values
Sex: 2 unique values
Age: 88 unique values
SibSp: 7 unique values
Parch: 7 unique values
Ticket: 681 unique values
Fare: 248 unique values
Cabin: 147 unique values
Embarked: 3 unique values
________________________________________________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin     

In [42]:
df1 = train.drop(['PassengerId', 'Ticket', 'Cabin'], axis = 'columns')

In [43]:
df1

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
886,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,13.0000,S
887,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,30.0000,S
888,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,23.4500,S
889,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,30.0000,C


In [91]:
# Crate FamilySize column from SibSp and Parch
df2= df1.copy()
df2['FamilySize'] = df1['SibSp'] + df1['Parch'] + 1

In [92]:
df3= df2.drop(['SibSp', 'Parch'],axis=1)

In [93]:
df3

,Survived,Pclass,Name,Sex,Age,Fare,Embarked,FamilySize
0,0,3,"Braund, Mr. Owen Harris",male,22.0,7.2500,S,2
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,71.2833,C,2
2,1,3,"Heikkinen, Miss. Laina",female,26.0,7.9250,S,1
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,53.1000,S,2
4,0,3,"Allen, Mr. William Henry",male,35.0,8.0500,S,1
...,...,...,...,...,...,...,...,...
886,0,2,"Montvila, Rev. Juozas",male,27.0,13.0000,S,1
887,1,1,"Graham, Miss. Margaret Edith",female,19.0,30.0000,S,1
888,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,23.4500,S,4
889,1,1,"Behr, Mr. Karl Howell",male,26.0,30.0000,C,1


In [94]:
# Create Title column from Name
df4= df3.copy()
df4['Title'] = df3['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(df4['Title'], df4['Sex'])

Sex,female,male
Title,,
Capt,0,1
Col,0,2
Countess,1,0
Don,0,1
Dr,1,6
Jonkheer,0,1
Lady,1,0
Major,0,2
Master,0,40


In [95]:
# Replace titles to more common names and group rare titles
common = ['Master', 'Mr', 'Miss', 'Mrs']
df4['Title'] = df4['Title'].replace('Mlle', 'Miss')
df4['Title'] = df4['Title'].replace('Ms', 'Miss')
df4['Title'] = df4['Title'].replace('Mme', 'Mrs')
df4['Title'] = [x if x in common else 'Rare' for x in df4['Title']]

df4['Title'].value_counts()

Mr        517
Miss      185
Mrs       126
Master     40
Rare       23
Name: Title, dtype: int64

In [96]:
title_ohe1 = pd.get_dummies(df4['Title'], prefix = 'Title', drop_first = True)
df5 = pd.concat([df4.drop('Title', axis = 1), title_ohe1], axis = 1)

In [97]:
df5

,Survived,Pclass,Name,Sex,Age,Fare,Embarked,FamilySize,Title_Miss,Title_Mr,Title_Mrs,Title_Rare
0,0,3,"Braund, Mr. Owen Harris",male,22.0,7.2500,S,2,0,1,0,0
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,71.2833,C,2,0,0,1,0
2,1,3,"Heikkinen, Miss. Laina",female,26.0,7.9250,S,1,1,0,0,0
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,53.1000,S,2,0,0,1,0
4,0,3,"Allen, Mr. William Henry",male,35.0,8.0500,S,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,"Montvila, Rev. Juozas",male,27.0,13.0000,S,1,0,0,0,1
887,1,1,"Graham, Miss. Margaret Edith",female,19.0,30.0000,S,1,1,0,0,0
888,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,23.4500,S,4,1,0,0,0
889,1,1,"Behr, Mr. Karl Howell",male,26.0,30.0000,C,1,0,1,0,0


In [98]:
df5= df5.drop(['Name'],axis=1)
df5['Sex'] = df4['Sex'].map({'female': 0, 'male': 1})

In [99]:
df5

,Survived,Pclass,Sex,Age,Fare,Embarked,FamilySize,Title_Miss,Title_Mr,Title_Mrs,Title_Rare
0,0,3,1,22.0,7.2500,S,2,0,1,0,0
1,1,1,0,38.0,71.2833,C,2,0,0,1,0
2,1,3,0,26.0,7.9250,S,1,1,0,0,0
3,1,1,0,35.0,53.1000,S,2,0,0,1,0
4,0,3,1,35.0,8.0500,S,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,13.0000,S,1,0,0,0,1
887,1,1,0,19.0,30.0000,S,1,1,0,0,0
888,0,3,0,NaN,23.4500,S,4,1,0,0,0
889,1,1,1,26.0,30.0000,C,1,0,1,0,0


In [100]:
# Check correlation of Age with other variables
age_corr = train.corr().abs().unstack().sort_values(kind="quicksort", ascending=False).reset_index()
age_corr.rename(columns={"level_0": "Feature 1", "level_1": "Feature 2", 0: 'Correlation Coefficient'}, inplace=True)
age_corr[age_corr['Feature 1'] == 'Age']

,Feature 1,Feature 2,Correlation Coefficient
5,Age,Age,1.000000
12,Age,Pclass,0.369226
16,Age,SibSp,0.308247
21,Age,Parch,0.189119
26,Age,Fare,0.096067
32,Age,Survived,0.077221
36,Age,PassengerId,0.036847


In [101]:
combine=[df5,df4]

In [102]:
# Impute Age based on Pclass
for i in range(0, 2):
    for j in range(0, 3):
        impute_df = df5[(df5['Sex'] == i) & \
                                (df5['Pclass'] == j+1)]['Age'].dropna()
        impute_ages[i,j] = int(impute_df.median())
            
for i in range(0, 2):
    for j in range(0, 3):
        df5.loc[ (df5.Age.isnull()) & (df5.Sex == i) & (df5.Pclass == j+1), 'Age'] = impute_ages[i,j]

In [103]:
df5

,Survived,Pclass,Sex,Age,Fare,Embarked,FamilySize,Title_Miss,Title_Mr,Title_Mrs,Title_Rare
0,0,3,1,22.0,7.2500,S,2,0,1,0,0
1,1,1,0,38.0,71.2833,C,2,0,0,1,0
2,1,3,0,26.0,7.9250,S,1,1,0,0,0
3,1,1,0,35.0,53.1000,S,2,0,0,1,0
4,0,3,1,35.0,8.0500,S,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,13.0000,S,1,0,0,0,1
887,1,1,0,19.0,30.0000,S,1,1,0,0,0
888,0,3,0,21.0,23.4500,S,4,1,0,0,0
889,1,1,1,26.0,30.0000,C,1,0,1,0,0


In [120]:
df6= df5.copy()

In [121]:
impute_embarked= ['', '', '']

for i in range(0, 3):
    impute_val = df6[df6['Pclass'] == i+1]['Embarked'].dropna().mode()[0]
    impute_embarked[i] = impute_val
        
for i in range(0, 3):
    df6.loc[ (df6.Embarked.isnull()) & (df6.Pclass == i+1), 'Embarked'] = impute_embarked[i]

In [122]:
print(df6.isnull().sum())

Survived      0
Pclass        0
Sex           0
Age           0
Fare          0
Embarked      0
FamilySize    0
Title_Miss    0
Title_Mr      0
Title_Mrs     0
Title_Rare    0
dtype: int64


In [123]:
embarked_ohe1 = pd.get_dummies(df6['Embarked'], prefix = 'Embarked', drop_first = True)
df7 = pd.concat([df6.drop('Embarked', axis = 1), embarked_ohe1], axis = 1)

In [124]:
df7

,Survived,Pclass,Sex,Age,Fare,FamilySize,Title_Miss,Title_Mr,Title_Mrs,Title_Rare,Embarked_Q,Embarked_S
0,0,3,1,22.0,7.2500,2,0,1,0,0,0,1
1,1,1,0,38.0,71.2833,2,0,0,1,0,0,0
2,1,3,0,26.0,7.9250,1,1,0,0,0,0,1
3,1,1,0,35.0,53.1000,2,0,0,1,0,0,1
4,0,3,1,35.0,8.0500,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,13.0000,1,0,0,0,1,0,1
887,1,1,0,19.0,30.0000,1,1,0,0,0,0,1
888,0,3,0,21.0,23.4500,4,1,0,0,0,0,1
889,1,1,1,26.0,30.0000,1,0,1,0,0,0,0


In [127]:
df7.to_csv('Clean_train.csv')